In [1]:
from dotenv import load_dotenv

import os, sys, random

sys.path.append("..")

from urllib.parse import urlparse

from sqlalchemy import create_engine
import pandas as pd

load_dotenv()

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)

import torch, sklearn, xgboost

In [2]:
db_url = urlparse(os.getenv("NEONDB_URL"))

DATA_PATH = os.getenv("DATA_PATH", "data")

MODEL_PATH = os.getenv("MODEL_PATH", "models")

LOGS_PATH = os.getenv("LOGS_PATH", "logs")

RESULTS_PATH = os.getenv("RESULTS_PATH", "results")

In [3]:

engine = create_engine(
    db_url.geturl(),
    connect_args={
        "sslmode": "require",
        "sslrootcert": os.getenv("NEONDB_SSLROOTCERT"),
    },
)


In [4]:
query = """
SELECT 
    s.id,
    s.session_id,
    s.user_id,
    s.choice,
    s.age,
    s.location,
    s.emotional_state,
    s.agitation_frequency,
    s.agitation_severity,
    s.family_visit_frequency,
    s.family_inconvenience,
    s.interaction_ability,
    s.functional_ability,
    s.behavior,
    s.affordability,
    s.eol_preference,
    s.crisis_type,
    s.crisis_intro,
    s.crisis_option_a,
    s.crisis_option_b,
    s.crisis_chance,
    s.crisis_wean,
    s.crisis_tube,
    s.crisis_comfort,
    s.resuscitation_chance,
    s.leave_hospital,
    s.internal_damage,
    s.future_arrest,
    s.family_preference,
    d.biological_sex,
    d.gender_identity,
    d.political_leaning,
    d.marital_status,
    d.religion,
    d.religious_importance,
    d.annual_income,
    d.education,
    d.family_history_dementia,
    d.personal_history_dementia,
    d.dementia_worry
FROM scenarios s
JOIN users u ON s.user_id = u.user_id
JOIN demographics d ON u.user_id = d.user_id
WHERE u.user_id NOT IN ('8NsBsQ7YBoDa', 'TEST123', 'LABTEST1', 'LABTEST2');

"""

df = pd.read_sql(query, engine)

In [5]:
df_clean = df.dropna(subset=['choice'])

In [6]:
cpr_option_quality = {
    'resuscitation_chance': [
        'low (around 9-in-100)', 
        'moderate (around 50-in-100)', 
        'moderate to high (around 70-in-100)'
    ],
    'leave_hospital': [
        'low (around 9-in-100)', 
        'moderate (around 50-in-100)', 
        'moderate to high (around 70-in-100)'
    ],
    'internal_damage': [
        'strong chance (around 80-in-100)', 
        'moderate to high chance (around 70-in-100)', 
        'moderate chance (around 50-in-100)'
    ],
    'future_arrest': [
        'strong chance (around 80-in-100)', 
        'moderate to high chance (around 70-in-100)', 
        'moderate chance (around 50-in-100)'
    ]
}

# Function to impute CPR features
def impute_cpr_features(row):
    if row['crisis_type'] == 'CPR':
        for feature, options in cpr_option_quality.items():
            if pd.isnull(row[feature]):
                if row['choice'] == 'treatment':
                    # Favor better options
                    row[feature] = random.choices(options, weights=[0.0, 0.0, 1])[0]
                else:
                    # Favor worse options
                    row[feature] = random.choices(options, weights=[0.1, 0.0, 0.0])[0]
    return row

# Apply and reassign — VERY IMPORTANT
df_filled = df_clean.apply(impute_cpr_features, axis=1)

In [7]:
# Vent options provided
vent_option_quality = {
    'crisis_chance': [
        'moderate chance (around 40-in-100)', 
        'strong chance (around 80-in-100)'
    ],
    'crisis_wean': [
        'by a few days (beyond how long you would live without the ventilator)',
        'by a few weeks (beyond how long you would live without the ventilator)',
        'by a few months (beyond how long you would live without the ventilator)',
        'indefinitely, until another problem arises'
    ],
    'crisis_tube': [
        'by a few days',
        'by a few weeks',
        'by a few months',
        'indefinitely, until another problem arises'
    ],
    'crisis_comfort': [
        'will die within hours to a few days',
        'will die within days to a few weeks'
    ]
}

# Step 1: Select only Vent rows with at least one null in the target columns
target_columns = ['crisis_chance', 'crisis_wean', 'crisis_tube', 'crisis_comfort']
vent_null_rows = df[
    (df['crisis_type'] == 'Vent') & 
    (df[target_columns].isnull().any(axis=1))
].copy()

# Step 2: Extraction function
def extract_and_fill(row):
    # Extract from crisis_option_a
    for col in ['crisis_chance', 'crisis_wean', 'crisis_tube', 'crisis_comfort']:
        if pd.isnull(row[col]):
            for option in vent_option_quality[col]:
                if option in row['crisis_option_a']:
                    row[col] = option
                    break  # Stop after finding first match

    # Extract from crisis_option_b
    for col in ['crisis_comfort']:
        if pd.isnull(row[col]):
            for option in vent_option_quality[col]:
                if option in row['crisis_option_b']:
                    row[col] = option
                    break  # Stop after finding first match

    return row

# Step 3: Apply extraction to affected rows
vent_null_rows = vent_null_rows.apply(extract_and_fill, axis=1)

# Step 4: Update original dataframe
df_filled.update(vent_null_rows)

print("✅ Missing values filled for Vent rows based on text matching.")


✅ Missing values filled for Vent rows based on text matching.


In [8]:
cpr_cols = ['resuscitation_chance', 'leave_hospital', 'internal_damage', 'future_arrest']
vent_cols = ['crisis_chance', 'crisis_wean', 'crisis_tube', 'crisis_comfort']

# Mask CPR features in ventilation rows
df_filled.loc[df_filled['crisis_type'] == 'Vent', cpr_cols] = 'MASKED'

# Mask Vent features in CPR rows
df_filled.loc[df_filled['crisis_type'] == 'CPR', vent_cols] = 'MASKED'

In [9]:
df_filled = df_filled.reset_index(drop=True)

In [11]:
df_filled.dropna(inplace=True)

In [12]:
df_filled.to_csv("data.csv", index=False)